# ECEMP case study

This Jupyter Notebook will guide you through ENBIOS preparation and use. A simple case study including onshore wind and open ground photovoltaic technologies in Spain was prepared. The necessary materials to follow this Notebook are here (https://drive.google.com/drive/folders/1YC6dtH-pPvL2UjO-z_udw6YKJ8lLBrZm?hl=es). 
The central file to run enbios is the BaseFile. You should have prepared your own BaseFile to follow the session. In case you didn't, you can follow the video instructions (XXXXX). An Example of how the BaseFile should look like at the end of the session is here (https://docs.google.com/spreadsheets/d/1uJFMEDOn_HlaJsQ8A1nz7oad0p6WxwJ5rzsQwRA39g8/edit?usp=sharing).
In case you have any doubts, you can always take a look at ENBIOS user manual (https://drive.google.com/file/d/1dTgWTgY2kTeZJF5Djl38xeymTJ90PMVa/view?usp=sharing).

### Step 1: Prepare the files you need to run enbios

This includes the following:
1. The BaseFile with all the required information.
2. A csv file containing the data of the energy produced by the technologies that define your system. Note that the technologies inclueded here have to match those introduced in the "Bare processor simulation" sheet in the Basefile. A csv file including onshore wind and openground photovoltaics has already been prepared for you here (https://drive.google.com/file/d/16qUilxqDXa-dVgWBdrK9VEdnxWOO3N4n/view?usp=sharing).
3. The latest version of the Ecoinvent Life Cycle Impact Assessment (LCIA) methods implemented in the ecoinvent database. It can be downlowded from Ecoinvent website: login -> Files -> Supporting documents -> LCIA_implementation.zip. Example: in 2021, the resulting downloaded file is called "LCIA_Implementation_v3.8.xlsx". In case you don't have access to Ecoinvent a copy of this file can be found here (https://docs.google.com/spreadsheets/d/1hGHdqtQRXQy_rhkDKN63S_w9mJXMLOkZ/edit?usp=sharing&ouid=107760011850830783238&rtpof=true&sd=true).
4. The spold files (.spold) of all the Ecoinvent processes to be used in your simulation. The spold files for onshore wind and openground photovoltaics in Spain are provided to you here (https://drive.google.com/drive/folders/1cvKhUMKoIrr0GLB8zUrjWu8B8M6wftGN?usp=sharing).

In [ ]:
from enbios.bin.script import Enbios
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

enbios = Enbios()

### Step 2: Run lcia_implementation_to_csv

This function prepares the Ecoinvent LCIA methods for their use in ENBIOS. (NOTE: this function has to be called only when Ecoinvent updates their database, usually once a year.)

**Input requirements** (to be specified in the cell below): 
1. lcia_implementation_path -> write the local path to the file LCIA_Implementation_v3.8.xlsx (use latest version).
2. method_out_path -> write the local path for the output csv file to be saved.

In [ ]:
lcia_implementation_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\LCIA_Implementation_v3.8.xlsx'
method_out_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\method_out.csv'

enbios.lcia_implementation_to_csv(lcia_implementation_path, method_out_path)

### Step 3: Run lci_to_nis

This function prepares an xlsx file of all interfaces, processors and combinations of these two.

**Input requirements** (to be specified in the cell below): 
1. basefile_url: BaseFile URL
2. spolds_folder: spold files folder of the Ecoinvent processes to be used in your simulation
3. lci_to_nis_output_path: write the local path for the output xlsx file to be saved.

In [ ]:
basefile_url = r'https://docs.google.com/spreadsheets/d/1703Qv1h8t3WFvwzdksfk-JfjIfqUnUZ1ba8NYfSijIk/edit#gid=387453521'
spolds_folder = 'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook'
lci_to_nis_output_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\lci_to_nis_output.xlsx'

enbios.lci_to_nis(basefile_url, spolds_folder, lci_to_nis_output_path)

### Step 4: Final BaseFile adjustments

Prior to running the simulation, it is important to vinculate results from step 2 and 3 with the BaseFile. In order to do so, upload the files resulting from step 2 and 3 in a Google Drive Folder and copy their respective links as follows:
1.  lcia_method_out.csv in "DatasetDef" Sheet, line 11.
2.  lci_to_nis.xlsx in "ImportCommandsLCIStructurals", line 2.

Moreover, the "ScalarIndicators" BaseFile sheet has still to be completed with the desired LCIA method indicators to be calculated. In the cell below, you can specify the LCIA method to be employed and **the output will provide an Excel sheet that can be directly pasted into the "ScalarIndicators" sheet**. To start, we propose you to use the method "ReCiPe Midpoint (H) V1.13".

**Input requirements** (to be specified in the cell below): 
1. input_path: local path to the lcia_indicators_list.xlsx file. This file is provided here (https://docs.google.com/spreadsheets/d/1EcA8DQTbP2ftQ-JBjgPY-iVQMdTh2VxV/edit?usp=sharing&ouid=107760011850830783238&rtpof=true&sd=true), nevertheless it has to be downloaded and stored as a local file.
2. output_path: local path for the output xlsx file to be saved.
3. method: LCIA method to be employed. IMPORTANT: you have to write the LCIA method name plus an underscore at the end. Example: "ReCiPe Midpoint (H) V1.13_".

In [ ]:
input_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\lcia_indicators_list.xlsx'
output_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook'
method = "ReCiPe Midpoint (H) V1.13_"

df = pd.read_excel(output_path + '\lcia_indicators_list.xlsx')

df.drop('Description', axis=1, inplace=True)
df = df.loc[df["Indicator"].str.contains(method, regex=False)]
list_indicators = df["Indicator"].tolist()
for i in list_indicators:
    new_indicator_name = i.replace(method, "").replace(" ", "_")
    df["Indicator"].replace({i:new_indicator_name}, inplace=True)

df.to_excel(output_path + '\lcia_indicators_list_filtered.xlsx' ,index=False)


#Preparation of the necessary files with the LCIA methods and the corresponding units
xls = pd.ExcelFile(lcia_implementation_path)
df2 = pd.read_excel(xls, "Indicators")
df2['enbios'] = [str(x) + '_' + str(y) + '_' + str(z) for x, y,z in zip(df2['Method'], df2['Category'], df2['Indicator'])]

df3 = pd.read_excel(input_path)
mydi = dict(zip(df2.enbios,df2.Unit))  
for key in mydi.keys():
    key = str(key) 
    df3.loc[df3.Indicator == key,'Unit'] = mydi[key]

df3.to_excel(output_path + '\lcia_implementation_units.xlsx', index=False)

#Filter the chosen LCIA method
df = pd.read_excel(output_path + '\lcia_implementation_units.xlsx')

df.drop('Description', axis=1, inplace=True)
df = df.loc[df["Indicator"].str.contains(method, regex=False)]
list_indicators = df["Indicator"].tolist()
for i in list_indicators:
    new_indicator_name = i.replace(method, "").replace(" ", "_")
    df["Indicator"].replace({i:new_indicator_name}, inplace=True)

df.to_excel(output_path + '\lcia_implementation_units_filtered.xlsx' ,index=False)

The output file, called lcia_indicators_list.xlsx, provides an Excel sheet that can be directly pasted into the "ScalarIndicators" sheet. In case you just want to analyse some of these indicators, you can "mute" the undesired ones by writing a "#" symbol in column A. We porpose you, to start, muting all the indicators except for four. For example: Climate Change, Fossil Depletion, Natural Land Transformation and Water Depletion.

### Step 5: Run enviro

Now that all the preparatory files are already set, the actual simulation can start by calling the "enviro" function.

**Input requirements**: 

1. yaml_path: A .yaml file that specifies the Base File path and the simulation file path.

In [ ]:
yaml_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\case_study_ECEMP.yaml' 

enbios.enviro(yaml_path)

If you get an error such as “ERROR:root:No database connection defined (DB_CONNECTION_STRING), exiting now!”, try closing the BaseFile in your browser and run the cell again. If the problem keeps happening, you will have to run the simulation from the terminal. In this case, first, open Anaconda Navigator and activate "enbios" environment prior to opening the Terminal. Then, execute the code: enbios enviro yaml_filename (with the local path to your yaml file). For more details you can check Manual's page 25.

### Step 6: Results visualisation

This function provides a graphical visualisation of the results.

**Input requirements** (to be specified in the cell below): 
1. results_path: local path where the enbios results were stored. The name of the csv file generated by enbios should be: indicators.csv.
2. dendrogram_level: number (integer) corresponding to the dendrogram level you want to explore.
3. plot_output_path: write the local path for the output .png graphs to be saved.

In [ ]:
results_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\output\indicators.csv'
dendrogram_level = 0
plot_output_path = r'C:\PhD_ICTA_Miquel\congresses\ECEMP2022\Test_Jupyter_Notebook\output\plots'


def visualization (dendrogram_level, plot_output_path):
    """
    Params:
    ->dendogram_level: integer 
    ->plot_output_path: path to save the results
    """
    
    df = pd.read_csv(results_path, delimiter=',')
    #rename the columns. Create a standard names
    def rename_columns(df):
        a=df.columns.tolist()
        namecolumns=('Scenario','System','Period','Scope','Processor','Indicator','Value','Unit')
        counter=0
        for i in ddd:
            df.rename(columns={i:namecolumns[counter]},inplace=True)
            counter=counter+1 
    #include the units
    dfunit=pd.read_excel(output_path + '\lcia_implementation_units_filtered.xlsx')
    mydi=dict(zip(dfunit.Indicator,dfunit.Unit))
    for key in mydi.keys():
        key=str(key)
        df.loc[df.Indicator==key,'Unit']=mydi[key]
    # #Create a dendrogram lvl maker
    df = df.loc[~df["Processor"].str.contains("environment")]
    df["Dendrogram level"] = df["Processor"].str.count("\.")
    df.drop(df[df['Scope']=='Internal'].index, inplace=True)
    listindicatorr=df['Indicator'].unique().tolist()
    label=df['Unit'].unique().tolist()
    #define a new class to iterate over the dataframes
    class my_dictionary(dict):
        # __init__ function
        def __init__(self):
            self = dict()
        def add(self, key, value):
            self[key] = value
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #Crea dataframes per cada indicador. Utilitza my dictionary per guardar-los a un diccionari
    dict_obj = my_dictionary()
    for i in df.columns:
        nom_dataframes=[]
        listindicators=df['Indicator'].unique()
        listindicator=listindicators.tolist()    
    for i,g in df.groupby('Indicator'):
        globals()['df_'+str(i)]=g
        nom_dataframes.append(i)
        dict_obj.add(i,g)
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #visualització
    sns.set()
    sns.set_style({'axes.facecolor':'white', 'grid.color': '.8', 'font.family':'Times New Roman'})
    sns.set_context("paper")
    fig, axes = plt.subplots(3,2,figsize=(10,10),dpi=500, )
    plt.subplots_adjust(left=0.1, 
                    top=0.9, 
                    wspace=0.2, 
                    hspace=0.5)
    
    colors=['r','b','y','mediumaquamarine','navajowhite','pink',] #define the colors
    color= iter(colors)
    for (key, data), ax in zip(dict_obj.items(), axes.ravel()): #iterate plots and axis
        fig.suptitle('Environmental Impacts in level {}'.format(dendrogram_level))
        data=data.loc[data['Dendrogram level'] == dendrogram_level]
        #get the unit label
        unit=data['Unit'].unique().tolist()
        #plot
        sns.barplot(ax=ax,x='Scenario',y='Value',data=data,color=next(color),alpha=0.8,saturation=0.7).set(
        xlabel='',ylabel=str(unit[0]))
        
        title=str(key)
        ax.set_title(title)
    plt.savefig(plot_output_path +'\Plot_Result'+'.png',facecolor='w', dpi=500)

            
visualization(dendrogram_level,plot_output_path)